# LLMs for summarization and translation
Input and target are sequences.

## Summarization

Extractive summarization
> select and combine parts of the original text. BERT (encoder model), T5 (encoder / decoder)

Abstractive summarization
> generate a summary word by word (sequence to sequence)

In [1]:
from datasets import load_dataset

dataset = load_dataset("ILSUM/ILSUM-1.0", "English")
print(f"Features: {dataset['train'].column_names}")

/home/atroncos/anaconda3/envs/pt_llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating validation split: 100%|████████████████████████████████████████████████████████████████████████| 898/898 [00:00<00:00, 33032.10 examples/s]

Features: ['id', 'Article', 'Heading', 'Summary']


In [2]:
example = dataset['train'][21]
example['Article']

'This is how an Apple Watch saved a man\'s life after detecting accidentIt all started when Gabe Burdett was waiting for his father Bob at their pre-designated location for some mountain biking at the Riverside State Park when he received a text alert from his dad\'s Apple Watch, saying it had detected a "hard fall".Burdett, from city of Spokane in Washington State later received another update from the Watch, saying his father had reached Sacred Heart Medical Center."We drove straight there but he was gone when we arrived. I get another update from the Watch saying his location has changed with a map location of SHMC. Dad flipped his bike at the bottom of Doomsday, hit his head and was knocked out until sometime during the ambulance ride," Burdett wrote in a Facebook post.The Watch notified 911 with the location and within 30 minutes, emergency medical services (EMS) took the injured Bob to the hospital."If you own an Apple Watch, set up your hard fall detection, it\'s not just for wh

Interesting attributes for training:
* Article: full text
* Summary: ground truth for testing predictions

Load pre-trained LLM

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "t5-small"  # encoder/decoder model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

input_ids = tokenizer.encode(
    'summarize: ' + example['Article'], # specify task by adding a prefix
    return_tensors='pt', max_length=512, truncation=True
)

summary_ids = model.generate(input_ids, max_length=150)
summary = tokenizer.decode(
    summary_ids[0], skip_special_tokens=True)

print(summary)

/home/atroncos/anaconda3/envs/pt_llm/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


a man was waiting for his father when he received a text alert from his dad's apple watch. the watch notified 911 with the location and within 30 minutes, emergency medical services took the injured Bob to the hospital. the watch notified 911 with the location and within 30 minutes, emergency medical services took the injured Bob to the hospital.


## Another example of summarization

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

dataset = load_dataset("opinosis")
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/home/atroncos/anaconda3/envs/pt_llm/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for opinosis contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/opinosis
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating train split: 100%|████████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 4694.19 examples/s]


In [12]:
print(f"Number of instances: {len(dataset['train'])}")
# Show the names of features in the training fold of the dataset
print(f"Feature names: {dataset['train'].column_names}")

Number of instances: 51
Feature names: ['review_sents', 'summaries']


In [15]:
# Encode the input example, obtain the summary, and decode it
example = dataset['train'][-2]['review_sents']
input_ids = tokenizer.encode("summarize: " + example, return_tensors='pt', max_length=512, truncation=True)
summary_ids = model.generate(input_ids, max_length=150)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("\nOriginal text\n", example[:400])
print("\nSummary\n", summary)


Original text
 I bought the 8, gig Ipod Nano that has the built, in video camera .
  Itunes has an on, line store, where you may purchase and download music and videos which will install onto the ipod .
I have lots of music cd's and dvd's, so currently I'm just interested in storing some of my music and videos on the ipod so I can enjoy them on my vacation, and while at work .
There's a right way and wrong wa

Summary
 I bought the 8, gig Ipod Nano that has the built, in video camera. Itunes has an on, line store, where you may purchase and download music and videos which will install onto the ipod.


## Translation

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Helsinki-NLP/opus-mt-fr-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# input_seq = "Je suis le poinçonneur des Lilas, le gars qu'on croise et qu'on ne regarde pas."
input_seq = """Je revois la ville en fête et en délire
Suffoquant sous le soleil et sous la joie
Et j'entends dans la musique les cris, les rires
Qui éclatent et rebondissent autour de moi"""
input_ids = tokenizer.encode(input_seq, return_tensors='pt')
translated_ids = model.generate(input_ids)
translated_text = tokenizer.decode(
    translated_ids[0], skip_special_tokens=True
)
translated_text

'I see the city in celebration and delirium Suffocating under the sun and joy And I hear in the music the screams, the laughter that burst and bounce around me'